In [1]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature.scalers import ranged_scaler
from datetime import datetime, timedelta
#from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 

In [2]:
from tkgngc.embeddings import PretrainedTKGEmbeddingWithTimestamps
from tkgngc.data_processing import TKGNGCDataProcessor
from tkgngc.model import train_model

In [3]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [4]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [5]:
cats_rows_list = metadata.rows(named=True)
cats_df.head()

timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2023-01-01 00:00:00,0.0,0.571429,0.25,0.5,0.5,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688,0.0,0.0,0
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.40973,0.246024,0.14197,0.112819,0.550194,0.406691,0.0,0.0,1
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698,0.0,0.0,2
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.40671,0.0,0.0,3
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726,0.0,0.0,4


In [6]:
cats_df = cats_df.to_pandas()

In [7]:
potential_causes = metadata['root_cause'].unique().to_list()

In [8]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [9]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [10]:
train_df.head()

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [11]:
device = torch.device('cpu')

In [12]:
train_df.columns

Index(['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1'],
      dtype='object')

In [13]:
tkgnc_data = TKGNGCDataProcessor(train_df,device,num_timestamps=20, lags=1)

/r2/home/j.lowhorn/bayesian_rca/src/tkgngc/data_processing.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['time'] = self.data.index
/r2/home/j.lowhorn/bayesian_rca/src/tkgngc/data_processing.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(


In [14]:
pretrained_tkg = PretrainedTKGEmbeddingWithTimestamps(
    num_entities=int(tkgnc_data.entity_indices.max().item()+1),
    num_relations=int(tkgnc_data.relation_indices.max().item()+1),
    embedding_dim=17,
    num_timestamps=tkgnc_data.num_timestamps,
).to(device)

In [15]:
quads = (
    tkgnc_data.entity_indices[:-1],  # Head entities
    tkgnc_data.relation_indices,  # Relations
    tkgnc_data.entity_indices[1:],  # Tail entities (shifted example)
    tkgnc_data.timestamp_indices[:-1],  # Timestamps
)


In [16]:
#pretrained_tkg.pretrain(quads, learning_rate=0.01, epochs=1000)
#torch.save(pretrained_tkg.state_dict(), 'pretrained_tkg')
pretrained_tkg.load_state_dict(torch.load('pretrained_tkg'))

<All keys matched successfully>

In [17]:
from CARAT.data_processing import *
from CARAT.model import *

In [18]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [19]:
cols = list(train_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

num_nodes = len(train_df.columns)

new_metadata = []
A0 = get_adjacency(cols,causal_indices,non_causal_indices,num_nodes)    
BATCH_SIZE = 64

model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:

for i, row in enumerate(cats_rows_list):
    if i % 10:
        print('Model: '+ str(i))
    start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    anomaly = eval(row['affected'])[0]
    root_cause = row['root_cause']
    mod_df = test_df[(test_df.index>= start_time) & (test_df.index<= end_time)]
    sample_data = TKGNGCDataProcessor(mod_df,device,num_timestamps=20, lags=1)
    model_data = create_granger_gat_data(pretrained_tkg,sample_data)
    model_data.retrain_tkg()
    features = model_data.time_series_data
    model_name = 'model_category_'+str(row['category'])

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    dataset = GraphDataset(features,model_data.entity_emb,model_data.timestamp_emb)
    dataloader = DataLoader(dataset,batch_size = BATCH_SIZE, shuffle=False)
    train_causal_vae(model, optimizer, dataloader, A0, num_epochs=50)

    model.eval()
    with torch.no_grad():
        _, _, _, learned_adj_matrix = model(features,model_data.entity_emb,model_data.timestamp_emb, num_nodes=num_nodes)
        causal_graph = learned_adj_matrix.numpy()
    
    adj_data = pd.DataFrame(causal_graph,index=cols,columns=cols)
    candidates = adj_data[anomaly].sort_values(ascending=False)
    candidates = candidates[candidates.index.isin( potential_causes)]

    potential_cause1 = candidates.index[0]
    potential_cause2 = candidates.index[1]
    potential_cause3 = candidates.index[2]
    if root_cause == potential_cause1:
        row['cause_1'] = 1
    if root_cause == potential_cause2:
        row['cause_2'] = 1
    if root_cause == potential_cause3:
        row['cause_3'] = 1
    new_metadata.append(row)

    if i%5 == 0:
        print('Iteration #: ' + str(i))
    


/r2/home/j.lowhorn/bayesian_rca/src/tkgngc/data_processing.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['time'] = self.data.index
/r2/home/j.lowhorn/bayesian_rca/src/tkgngc/data_processing.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(


Epoch 0, Loss: 0.0012
Epoch 0: Loss = 164.7284
Epoch 10: Loss = 29.3025
Epoch 20: Loss = 23.8133
Epoch 30: Loss = 23.1906
Epoch 40: Loss = 22.1524


NameError: name 'x_batch' is not defined

In [ ]:
stats = pl.DataFrame(new_metadata)
agg_stats = stats.select(pl.sum("cause_1", "cause_2",'cause_3'))
agg_stats.select(pl.sum_horizontal(pl.all())).item()/stats.shape[0]